# Add Random Ground Truth and Predictions
This notebook intends to mimic the real situation where new ground truth data and the corresponding predictions get added once in a while. Here we randomly select a certain amount of input images, derive the predictions, and add both to the storage volume. 

In [1]:
!pip install dominate visdom numba

In [2]:
import os
import shutil
import random
import subprocess

In [3]:
volume_name = 'AdditionalDeepLIIFVolume'
num_files = 5

dir_source = f'/mnts/{volume_name}/DeepLIIF_Datasets/test'
dir_target = f'/mnts/{volume_name}/DeepLIIF_Datasets/model_eval'

In [4]:
fns = os.listdir(dir_source) # assume this directory has no sub-directories
fns_input = random.sample(fns,num_files)
print(fns_input)

['Lung_34000_34000_3_3.png', 'Lung_26000_34000_2_2.png', 'Lung_28000_32000_0_0.png', 'Lung_34000_26000_0_1.png', 'Lung_32000_32000_2_1.png']


In [5]:
dir_target_gt_tmp = f'{dir_target}/gt_images_tmp'
dir_target_pred_tmp = f'{dir_target}/model_images_tmp'

shutil.rmtree(dir_target_gt_tmp,ignore_errors=True)
shutil.rmtree(dir_target_pred_tmp,ignore_errors=True)

os.mkdir(dir_target_gt_tmp)
os.mkdir(dir_target_pred_tmp)

In [6]:
for fn in fns_input:
    shutil.copy2(f'{dir_source}/{fn}',dir_target_gt_tmp)

In [7]:
%%time
path_script = '/userfs/deepliif-repo/cli.py'
path_model = f'/mnts/{volume_name}/deepliif-ws-output/Test_Model_wendy_ws'

os.environ['DEEPLIIF_SEED'] = 'None'
os.environ['DEEPLIIF_MODEL_DIR'] = path_model
# out = subprocess.check_output(f'python {path_script} test --input-dir {dir_target_gt_tmp} --output-dir {dir_target_pred_tmp}',shell=True)
# print(out.decode())

CPU times: user 21 µs, sys: 5 µs, total: 26 µs
Wall time: 29.6 µs


In [8]:
%%time
!python /userfs/deepliif-repo/cli.py test --input-dir /mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/gt_images_tmp --output-dir /mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/model_images_tmp

Processing 5 images  [####################################]  100%                                    
CPU times: user 436 ms, sys: 78.9 ms, total: 515 ms
Wall time: 36.6 s


In [9]:
dir_target_gt = f'{dir_target}/gt_images'
dir_target_pred = f'{dir_target}/model_images'

In [10]:
print('Before:')
out = subprocess.check_output(f'ls -lh {dir_target_gt} | wc -l',shell=True)
print(dir_target_gt,out.decode())
out = subprocess.check_output(f'ls -lh {dir_target_pred} | wc -l',shell=True)
print(dir_target_pred,out.decode())

Before:
/mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/gt_images 6

/mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/model_images 41



In [11]:
for fn in os.listdir(dir_target_gt_tmp):
    shutil.copy2(f'{dir_target_gt_tmp}/{fn}',dir_target_gt)
    subprocess.run(f'touch {dir_target_gt}/{fn}',shell=True) # shutil.copy2 keeps the original timestamp
    
for fn in os.listdir(dir_target_pred_tmp):
    shutil.copy2(f'{dir_target_pred_tmp}/{fn}',dir_target_pred)

In [12]:
print('After:')
out = subprocess.check_output(f'ls -lh {dir_target_gt} | wc -l',shell=True)
print(dir_target_gt,out.decode())
out = subprocess.check_output(f'ls -lh {dir_target_pred} | wc -l',shell=True)
print(dir_target_pred,out.decode())

After:
/mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/gt_images 11

/mnts/AdditionalDeepLIIFVolume/DeepLIIF_Datasets/model_eval/model_images 81



In [13]:
shutil.rmtree(dir_target_gt_tmp,ignore_errors=True)
shutil.rmtree(dir_target_pred_tmp,ignore_errors=True)